# Main generator


In [ ]:
import random
from copy import deepcopy
from typing import Literal, Optional

import numpy as np

In [28]:
Layout = list[list[str]]


QWERTY_LOW_LAYOUT: Layout = [
    ["`", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "-", "=", "<back>"],
    ["<tab>", "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "[", "]", "\\"],
    ["<caps>", "a", "s", "d", "f", "g", "h", "j", "k", "l", ";", "'", "<enter>"],
    ["<shift>", "z", "x", "c", "v", "b", "n", "m", ",", ".", "/", "<shift>"],
    ["<win>", "<alt>", " ", " ", " ", " ", " ", " ", "<alt>", "<fn>"],
]

QWERTY_HIGH_LAYOUT: Layout = [
    ["~", "!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "+", "<back>"],
    ["<tab>", "Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P", "{", "}", "|"],
    ["<caps>", "A", "S", "D", "F", "G", "H", "J", "K", "L", ":", '"', "<enter>"],
    ["<shift>", "Z", "X", "C", "V", "B", "N", "M", "<", ">", "?", "<shift>"],
    ["<win>", "<alt>", " ", " ", " ", " ", " ", " ", "<alt>", "<fn>"],
]

In [ ]:
LogType = Literal["basic"] | Literal["debug"] | Literal["error"]


class Logger:
    def __init__(self, verbose: bool = True, hide_types: list[LogType] = []) -> None:
        self.verbose = verbose
        self.hide_types = set(hide_types)

    def log(self, message: str, log_type: LogType = "basic") -> None:
        if self.verbose and log_type not in self.hide_types:
            print(message)


LOGGER = Logger()

In [131]:
Position = tuple[int, int]


class Finger:
    def __init__(
        self, initial_position: Position, name: str, logger: Logger = LOGGER
    ) -> None:
        self.name = name
        self.initial_position = initial_position

        self.logger = logger

        self.reset()

        # Constants

        self.wait_before_return = 4  # in ticks

        self.long_row_move_shift = 3
        self.long_row_move_penalty = 1

        self.row_penalty_coefficient = 1
        self.column_penalty_coefficient = 1.2

    def reset(self):
        self.current_position = self.initial_position
        self.ticks_before_return = 0  # if == 0, returns to the initial position
        self.typed_keys = 0

    def move(self, position: Position):
        self.current_position = position

        self.ticks_before_return = self.wait_before_return
        self.typed_keys += 1

    def tick(self) -> float:
        if self.ticks_before_return > 0:
            self.ticks_before_return -= 1

        if self.ticks_before_return == 0:
            score = self.get_score(self.initial_position)
            self.current_position = self.initial_position
            return score

        return 0

    def get_score(self, target_position: Position) -> float:
        x1, y1 = self.current_position
        x2, y2 = target_position

        row_distance = abs(x1 - x2) ** 2
        column_distance = abs(y1 - y2) ** 2

        penalty = 0
        if row_distance > self.long_row_move_shift:
            penalty = self.long_row_move_penalty
        return (
            row_distance * self.row_penalty_coefficient
            + column_distance * self.column_penalty_coefficient
            + penalty
        )

    def show_statistics(self):
        self.logger.log(
            f"Name: {self.name}\t\
            Typed keys: {self.typed_keys}\t\
            Ticks before return: {self.ticks_before_return}\t\
            Current position: {self.current_position}\t\
            Default position: {self.initial_position}\n\n"
        )

In [ ]:
DEFAULT_FINGERS: list[Finger] = [
    Finger((2, 1), "левый мизинец"),
    Finger((2, 2), "левый безымянный"),
    Finger((2, 3), "левый средний"),
    Finger((2, 4), "левый указательный"),
    Finger((4, 3), "левый большой"),
    Finger((4, 6), "правый большой"),
    Finger((2, 7), "правый указательный"),
    Finger((2, 8), "правый средний"),
    Finger((2, 9), "правый безымянный"),
    Finger((2, 10), "правый мизинец"),
]

SwapType = Literal["low_layout"] | Literal["high_layout"] | Literal["between_layouts"]


class KeyboardLayout:
    @staticmethod
    def layout_to_dict(layout: Layout) -> dict[str, list[Position]]:
        layout_dict: dict[str, list[Position]] = {}

        for i in range(len(layout)):
            for j in range(len(layout[i])):
                button = layout[i][j]
                if button in layout_dict:
                    layout_dict[button].append((i, j))
                else:
                    layout_dict[button] = [(i, j)]

        return layout_dict

    def _finish_move(self):
        for finger in self.fingers:
            self.total_score += finger.tick()

    def __init__(self, low_layout: Layout, high_layout: Layout, logger: Logger = LOGGER):
        self.low_layout = deepcopy(low_layout)
        self.high_layout = deepcopy(high_layout)

        self.low_layout_dict = KeyboardLayout.layout_to_dict(self.low_layout)
        self.high_layout_dict = KeyboardLayout.layout_to_dict(self.high_layout)

        self.logger = logger

        self.fingers = deepcopy(DEFAULT_FINGERS)

        self.reset()

    def reset(self):
        self.total_score = 0
        for f in self.fingers:
            f.reset()

    def move_one_finger(
        self, positions: list[Position], busy_finger_id: Optional[int] = None
    ) -> tuple[tuple[int, Position], float]:
        best_finger_id: int = 0
        best_score = np.inf

        final_position: Position = (0, 0)

        for position in positions:
            scores = [
                finger.get_score(position) if i != busy_finger_id else np.inf
                for i, finger in enumerate(self.fingers)
            ]

            candidate_finger_id = int(np.argmin(scores))
            candidate_score = scores[candidate_finger_id]

            if candidate_score < best_score:
                best_score = candidate_score
                best_finger_id = candidate_finger_id
                final_position = position

        return (best_finger_id, final_position), best_score

    def move_two_fingers(
        self, positions: list[Position]
    ) -> tuple[tuple[int, Position], tuple[int, Position], float]:
        shift_positions = self.low_layout_dict["<shift>"]

        # firstly reach SHIFT, then - positions
        finger_shift_info_1, shift_distance_1 = self.move_one_finger(shift_positions)
        finger_btn_info_1, d1_btn = self.move_one_finger(
            positions, finger_shift_info_1[0]
        )
        total_distance_1 = shift_distance_1 + d1_btn

        # firstly reach positions, then - SHIFT
        finger_btn_info_2, d1_btn = self.move_one_finger(positions)
        finger_shift_info_2, shift_distance_2 = self.move_one_finger(
            shift_positions, finger_btn_info_2[0]
        )
        total_distance_2 = shift_distance_2 + d1_btn

        if total_distance_1 < total_distance_2:
            return finger_btn_info_1, finger_shift_info_1, total_distance_1

        return finger_btn_info_2, finger_shift_info_2, total_distance_2

    def find_button(self, button: str):
        if button in self.low_layout_dict:
            (finger_id, finger_position), score = self.move_one_finger(
                self.low_layout_dict[button]
            )

            self.fingers[finger_id].move(finger_position)
            self.total_score += score

            self.logger.log(f"{button}:\t{self.fingers[finger_id].name}")

        elif button in self.high_layout_dict:
            (
                (finger_id_1, finger_position_1),
                (finger_id_2, finger_position_2),
                score,
            ) = self.move_two_fingers(self.high_layout_dict[button])

            self.fingers[finger_id_1].move(finger_position_1)
            self.fingers[finger_id_2].move(finger_position_2)
            self.total_score += score

            self.logger.log(
                f"{button}:\t{self.fingers[finger_id_1].name} + {self.fingers[finger_id_2].name}"
            )

        else:
            self.logger.log(f"NO SUCH KEY: {button}")

        self._finish_move()

    def type_text(self, text: list[str]) -> float:
        for button in text:
            self.find_button(button)

        return self.total_score

    def swap_buttons(self, position1: Position, position2: Position, swap_type: SwapType):
        if swap_type == "high_layout":
            layout_from = layout_to = self.high_layout
            layout_from_dict = layout_to_dict = self.high_layout_dict
        elif swap_type == "low_layout":
            layout_from = layout_to = self.low_layout
            layout_from_dict = layout_to_dict = self.low_layout_dict
        else:  # swap_type == "between_layouts"
            layout_from = self.low_layout
            layout_to = self.high_layout
            layout_from_dict = self.low_layout_dict
            layout_to_dict = self.high_layout_dict

        x1, y1 = position1
        btn1 = layout_from[x1][y1]
        x2, y2 = position2
        btn2 = layout_to[x2][y2]

        layout_from[x1][y1], layout_to[x2][y2] = layout_to[x2][y2], layout_from[x1][y1]

        layout_from_dict[btn1].remove(position1)
        layout_from_dict[btn1].append(position2)

        layout_to_dict[btn2].append(position1)
        layout_to_dict[btn2].remove(position2)

    def show_statistics(self):
        self.logger.log("\nStatistics:")
        for f in self.fingers:
            f.show_statistics()

In [ ]:
def get_buttons_set() -> set[str]:
    buttons: set[str] = set()

    for layout in [QWERTY_LOW_LAYOUT, QWERTY_HIGH_LAYOUT]:
        for i in range(len(layout)):
            for btn in layout[i]:
                buttons.add(btn)

    return buttons


BUTTONS_SET = get_buttons_set()
KEYBOARD_LAYOUT_SHAPE = (len(QWERTY_HIGH_LAYOUT), len(QWERTY_HIGH_LAYOUT[0]))

In [133]:
def generate_cell(keyboard_shape):
    x = random.randint(0, len(keyboard_shape) - 1)
    y = random.randint(0, keyboard_shape[x] - 1)
    return x, y


def generate_layout(all_keys, keyboard_shape):
    low = [[0 for _ in range(row)] for row in keyboard_shape]
    high = [[0 for _ in range(row)] for row in keyboard_shape]
    x1, y1 = generate_cell(keyboard_shape)
    x2, y2 = x1, y1
    while x1 == x2 and y1 == y2:
        x2, y2 = generate_cell(keyboard_shape)
    low[x1][y1] = "<shift>"
    high[x1][y1] = "<shift>"
    low[x2][y2] = "<shift>"
    high[x2][y2] = "<shift>"

    temp = deepcopy(all_keys)
    random.shuffle(temp)

    for i in range(len(low)):
        for j in range(len(low[i])):
            key = low[i][j]
            if key == "<shift>":
                continue
            key = temp.pop()
            low[i][j] = key

    for i in range(len(high)):
        for j in range(len(high[i])):
            key = high[i][j]
            if key == "<shift>":
                continue
            key = temp.pop()
            high[i][j] = key

    return low, high


generate_layout(all_keys, keyboard_shape)

([['*', '*', '_', 'V', '\n', 'B', ' ', 'X', 'N', 'B', '_', 'A', ' ', '<fn>'],
  ['S',
   '}',
   '!',
   'I',
   'L',
   ' ',
   '<caps>',
   '#',
   'K',
   '<win>',
   '|',
   'G',
   ' ',
   '?'],
  [' ', ' ', 'V', 'S', 'T', '<back>', '}', '<shift>', '!', 'O', 'L', 'W', '{'],
  ['U',
   'P',
   'M',
   '<alt>',
   'O',
   '<caps>',
   '\t',
   '<alt>',
   '"',
   '<win>',
   'N',
   '<'],
  ['<shift>', ' ', 'W', '@', '@', 'C', '$', 'F', '(', 'Q']],
 [['?', ')', '%', ' ', 'E', 'U', 'Q', '|', 'G', 'Y', '%', ' ', 'D', 'R'],
  ['P', '+', 'T', '^', '#', ')', '$', ':', ' ', '\n', 'H', 'Z', '+', '&'],
  [':', 'Y', '<', '~', '&', '^', '>', '<shift>', 'H', 'Z', 'F', '<alt>', 'D'],
  ['"', 'A', '<back>', 'R', '<fn>', '~', '\t', '<alt>', 'E', 'J', 'M', 'X'],
  ['<shift>', ' ', '>', 'K', '{', 'C', '(', ' ', 'J', 'I']])

In [134]:
len(all_keys)

122

In [135]:
keyboard_shape

[14, 14, 13, 12, 10]

In [136]:
k = keyboard(QWERTY_LOW_LAYOUT, high_layout, low_layout_dict, high_dict, show=True)
k.low_dict["s"], k.low_dict["q"], k.high_dict["S"], k.high_dict["Q"]

([[2, 2]], [[1, 1]], [[2, 2]], [[1, 1]])

In [137]:
k.swap_cells_low([1, 1], [2, 2])
k.swap_cells_high([1, 1], [2, 2])

In [138]:
k.low_dict["s"], k.low_dict["q"], k.high_dict["S"], k.high_dict["Q"]

([[1, 1]], [[2, 2]], [[1, 1]], [[2, 2]])

In [139]:
k.keyboard_low

[['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', '<back>'],
 ['\t', 's', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', '\\'],
 ['<caps>', 'a', 'q', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'", '\n'],
 ['<shift>', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [140]:
k.keyboard_high

[['~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '<back>'],
 ['\t', 'S', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '{', '}', '|'],
 ['<caps>', 'A', 'Q', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ':', '"', '\n'],
 ['<shift>', 'Z', 'X', 'C', 'V', 'B', 'N', 'M', '<', '>', '?', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [141]:
k.low_dict["w"], k.high_dict["E"]

([[1, 2]], [[1, 3]])

In [142]:
k.swap_cells_diff([1, 2], [1, 3])
k.keyboard_low

[['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', '<back>'],
 ['\t', 's', 'E', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', '\\'],
 ['<caps>', 'a', 'q', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'", '\n'],
 ['<shift>', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [143]:
k.keyboard_high

[['~', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+', '<back>'],
 ['\t', 'S', 'W', 'w', 'R', 'T', 'Y', 'U', 'I', 'O', 'P', '{', '}', '|'],
 ['<caps>', 'A', 'Q', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ':', '"', '\n'],
 ['<shift>', 'Z', 'X', 'C', 'V', 'B', 'N', 'M', '<', '>', '?', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

In [144]:
(
    k.low_dict["w"],
    k.high_dict["E"],
    k.high_dict["w"],
    k.low_dict["E"],
)

([], [], [[1, 3]], [[1, 2]])

In [145]:
k.show_fingers_stat()

Name: левый мизинец		 Typed keys: 0	 Current counter: 0	 Current position: [2, 1]	 Default position: [2, 1]
Name: левый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 2]	 Default position: [2, 2]
Name: левый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 3]	 Default position: [2, 3]
Name: левый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 4]	 Default position: [2, 4]
Name: левый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 3]	 Default position: [4, 3]
Name: правый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 6]	 Default position: [4, 6]
Name: правый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 7]	 Default position: [2, 7]
Name: правый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 8]	 Default position: [2, 8]
Name: правый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 9]	 Default position: [2, 9]
Name: пр

In [146]:
k.type_string("Hi my name is Seva!")

H: правый указательный+левый мизинец
i: правый средний
 : левый большой
m: правый большой
y: правый указательный
 : левый большой
n: правый большой
a: левый мизинец
m: правый указательный
e: левый средний
 : левый большой
i: правый средний
s: левый мизинец
 : левый большой
S: левый мизинец+правый мизинец
e: левый средний
v: левый указательный
a: левый мизинец
!: левый мизинец+правый мизинец


32.096399481450646

In [147]:
k.show_fingers_stat()

Name: левый мизинец		 Typed keys: 6	 Current counter: 3	 Current position: [0, 1]	 Default position: [2, 1]
Name: левый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 2]	 Default position: [2, 2]
Name: левый средний		 Typed keys: 2	 Current counter: 0	 Current position: [2, 3]	 Default position: [2, 3]
Name: левый указательный		 Typed keys: 1	 Current counter: 1	 Current position: [3, 4]	 Default position: [2, 4]
Name: левый большой		 Typed keys: 4	 Current counter: 0	 Current position: [4, 3]	 Default position: [4, 3]
Name: правый большой		 Typed keys: 2	 Current counter: 0	 Current position: [4, 6]	 Default position: [4, 6]
Name: правый указательный		 Typed keys: 3	 Current counter: 0	 Current position: [2, 7]	 Default position: [2, 7]
Name: правый средний		 Typed keys: 2	 Current counter: 0	 Current position: [2, 8]	 Default position: [2, 8]
Name: правый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 9]	 Default position: [2, 9]
Name: пр

In [148]:
k.reset_keyboard()
k.show_fingers_stat()

Name: левый мизинец		 Typed keys: 0	 Current counter: 0	 Current position: [2, 1]	 Default position: [2, 1]
Name: левый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 2]	 Default position: [2, 2]
Name: левый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 3]	 Default position: [2, 3]
Name: левый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 4]	 Default position: [2, 4]
Name: левый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 3]	 Default position: [4, 3]
Name: правый большой		 Typed keys: 0	 Current counter: 0	 Current position: [4, 6]	 Default position: [4, 6]
Name: правый указательный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 7]	 Default position: [2, 7]
Name: правый средний		 Typed keys: 0	 Current counter: 0	 Current position: [2, 8]	 Default position: [2, 8]
Name: правый безымянный		 Typed keys: 0	 Current counter: 0	 Current position: [2, 9]	 Default position: [2, 9]
Name: пр

As we can see, we successfully can swap different keys in given keyboard and calculate distance function fro given string.


# Check

let's check on some program from this [dataset](https://www.kaggle.com/datasets/joshuwamiller/software-code-dataset-cc).


In [14]:
import pandas as pd

df = pd.read_csv("./validated/200717.cpp.csv", sep="`")
temp = df["code"]
len(temp)

14

In [15]:
temp_sent = "\n".join(temp)

In [16]:
temp_sent

'void update(bool flag[DIM][DIM], string type, string pos) {\n\tint x=pos[0]-\'a\', y=pos[1]-\'1\';\n\tif(type=="rook") {\n\t\tfor (int i=0; i<DIM; i++) {\n\t\t\tfor (int j=0; j<DIM; j++) {\n\t\t\t\tif((i==x || j==y) && !(i==x && j==y)) flag[i][j]=true;\n\t\tfor (int i=-1; i<=1; i++) {\n\t\t\tfor (int j=-1; j<=1; j++) {\n\t\t\t\tif(!(i==0 && j==0)) {\n\t\t\t\t\tint u=x+i, v=y+j;\n\t\t\t\t\tif(0<=u && u<DIM && 0<=v && v<DIM) flag[u][v]=true;\nint main() {\n\tbool flag[DIM][DIM];\n\tfor (int i=0; i<DIM; i++) for (int j=0; j<DIM; j++) flag[i][j]=false;'

In [17]:
k1 = keyboard(QWERTY_LOW_LAYOUT, high_layout, low_layout_dict, high_dict)
k1.type_string(temp_sent)

972.7854054794448

In [18]:
k1.keyboard_low

[['`', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '-', '=', '<back>'],
 ['\t', 'q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']', '\\'],
 ['<caps>', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', "'", '\n'],
 ['<shift>', 'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/', '<shift>'],
 ['<win>', '<alt>', ' ', ' ', ' ', ' ', ' ', ' ', '<alt>', '<fn>']]

Everything works


## Build cpp dataset


In [ ]:
import os

from tqdm import tqdm

count = 0
directory = "./validated"
sentences = []
for filename in tqdm(os.listdir(directory)):
    f = os.path.join(directory, filename)
    df = pd.read_csv(f, sep="`")
    temp = df["code"]
    temp_sent = "\n".join(temp)
    sentences.append(temp_sent)
    count += 1

In [ ]:
cpp_programs = pd.DataFrame(sentences)
cpp_programs.head()

,0
0,"point operator-(point p1,point p2)\n p1.x-=p2..."
1,"int main() {\n map<char, int> m;\n m['6'..."
2,"int main()\n\tint n,s[1001],cnt=0;\n\tscanf(""%..."
3,int main() {\n\tcin >> n;\n\tfor (int i = 0; i...
4,"int main()\n char a, b, c, d, kozir;\n ..."


In [ ]:
cpp_programs.columns = ["text"]

In [ ]:
cpp_programs.head()

,text
0,"point operator-(point p1,point p2)\n p1.x-=p2..."
1,"int main() {\n map<char, int> m;\n m['6'..."
2,"int main()\n\tint n,s[1001],cnt=0;\n\tscanf(""%..."
3,int main() {\n\tcin >> n;\n\tfor (int i = 0; i...
4,"int main()\n char a, b, c, d, kozir;\n ..."


In [ ]:
len(sentences), len(cpp_programs)

(119989, 119989)

In [ ]:
cpp_programs.to_csv("./cpp_programs.csv")

In [ ]:
test_df = pd.read_csv("./cpp_programs.csv", index_col=0)
test_df.head()

,text
0,"point operator-(point p1,point p2)\n p1.x-=p2..."
1,"int main() {\n map<char, int> m;\n m['6'..."
2,"int main()\n\tint n,s[1001],cnt=0;\n\tscanf(""%..."
3,int main() {\n\tcin >> n;\n\tfor (int i = 0; i...
4,"int main()\n char a, b, c, d, kozir;\n ..."
